In [1]:
class MNISTLoader():
    """数据获取及预处理 tf.keras.datasets"""
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)  # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1) # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
    
    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return  self.train_data[index, :], self.train_label[index]

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()   # tensorflow v2 使用 v1 的语句
import numpy as np
import os

"""配置GPU分配"""
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  #选择哪一块gpu,如果是-1，就是调用cpu
config = tf.ConfigProto() #对session进行参数配置
config.allow_soft_placement=True #如果你指定的设备不存在，允许TF自动分配设备
#config.gpu_options.per_process_gpu_memory_fraction=0.6 #分配百分之七十的显存给程序使用，避免内存溢出，可以自己调整
#config.gpu_options.allow_growth = True #按需分配显存，这个比较重要

"""CNN模型"""

class CNN(tf.keras.Model):
    
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        output = tf.nn.softmax(x)
        return output

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
"""模型的训练: tf.keras.losses, tf.keras.optimizer"""

tf.enable_eager_execution()

"""定义一些模型超参数"""
num_epochs = 5
batch_size = 50
learning_rate = 0.001

"""实例化模型和数据读取类，并实例化一个 tf.keras.optimizer 的优化器（这里使用常用的 Adam 优化器）"""
model = CNN()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# 然后迭代进行以下步骤：
# 1. 从 DataLoader 中随机取一批训练数据；
# 2. 将这批数据送入模型，计算出模型的预测值；
# 3. 将模型预测值与真实值进行比较，计算损失函数（loss）。这里使用 tf.keras.losses 中的交叉熵函数作为损失函数；
# 4. 计算损失函数关于模型变量的导数；
# 5. 将求出的导数值传入优化器，使用优化器的 apply_gradients 方法更新模型参数以最小化损失函数。

num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.296363
batch 1: loss 2.270715
batch 2: loss 2.187877
batch 3: loss 1.982385
batch 4: loss 1.807245
batch 5: loss 1.845409
batch 6: loss 1.597108
batch 7: loss 1.368009
batch 8: loss 1.067334
batch 9: loss 1.270214
batch 10: loss 0.999591
batch 11: loss 0.865573
batch 12: loss 0.661873
batch 13: loss 0.933976
batch 14: loss 0.754275
batch 15: loss 0.657850
batch 16: loss 0.676627
batch 17: loss 0.566160
batch 18: loss 0.802166
batch 19: loss 0.446005
batch 20: loss 0.469092
batch 21: loss 0.633505
batch 22: loss 0.439349
batch 23: loss 0.311789
batch 24: loss 0.237642
batch 25: loss 0.467201
batch 26: loss 0.418399
batch 27: loss 0.531250
batch 28: loss 0.519104
batch 29: loss 0.552959
batch 30: loss 0.242774
batch 31: loss 0.229179
batch 32: loss 0.586027
batch 33: loss 0.288305
batch 34: loss 0.299979
batch 35: loss 0.221953
batch 36: loss 0.204748
batch 37: loss 0.397683
batch 38: loss 0.276823
batch 39: loss 0.190378
batch 40: loss 0.277534
batch 41: loss 0.396031
ba

batch 334: loss 0.013921
batch 335: loss 0.066979
batch 336: loss 0.090331
batch 337: loss 0.200180
batch 338: loss 0.054573
batch 339: loss 0.103745
batch 340: loss 0.014072
batch 341: loss 0.167941
batch 342: loss 0.051120
batch 343: loss 0.051026
batch 344: loss 0.095992
batch 345: loss 0.047537
batch 346: loss 0.045250
batch 347: loss 0.042051
batch 348: loss 0.019334
batch 349: loss 0.122187
batch 350: loss 0.063892
batch 351: loss 0.101734
batch 352: loss 0.044172
batch 353: loss 0.023057
batch 354: loss 0.027752
batch 355: loss 0.037557
batch 356: loss 0.020930
batch 357: loss 0.140037
batch 358: loss 0.013660
batch 359: loss 0.003904
batch 360: loss 0.008538
batch 361: loss 0.011240
batch 362: loss 0.030363
batch 363: loss 0.061434
batch 364: loss 0.048957
batch 365: loss 0.052309
batch 366: loss 0.026148
batch 367: loss 0.195397
batch 368: loss 0.038683
batch 369: loss 0.044517
batch 370: loss 0.024629
batch 371: loss 0.060239
batch 372: loss 0.104414
batch 373: loss 0.052056


batch 664: loss 0.156759
batch 665: loss 0.069136
batch 666: loss 0.019159
batch 667: loss 0.011316
batch 668: loss 0.109090
batch 669: loss 0.036558
batch 670: loss 0.010826
batch 671: loss 0.077242
batch 672: loss 0.029458
batch 673: loss 0.040006
batch 674: loss 0.015333
batch 675: loss 0.137918
batch 676: loss 0.005742
batch 677: loss 0.019658
batch 678: loss 0.003372
batch 679: loss 0.006132
batch 680: loss 0.003853
batch 681: loss 0.006579
batch 682: loss 0.052542
batch 683: loss 0.008997
batch 684: loss 0.005453
batch 685: loss 0.002977
batch 686: loss 0.017676
batch 687: loss 0.010257
batch 688: loss 0.026917
batch 689: loss 0.113061
batch 690: loss 0.110806
batch 691: loss 0.017521
batch 692: loss 0.016926
batch 693: loss 0.014432
batch 694: loss 0.036764
batch 695: loss 0.050215
batch 696: loss 0.043072
batch 697: loss 0.260026
batch 698: loss 0.067183
batch 699: loss 0.063695
batch 700: loss 0.072481
batch 701: loss 0.097196
batch 702: loss 0.015631
batch 703: loss 0.215543


batch 992: loss 0.028220
batch 993: loss 0.060280
batch 994: loss 0.003750
batch 995: loss 0.004408
batch 996: loss 0.002470
batch 997: loss 0.029866
batch 998: loss 0.062758
batch 999: loss 0.155765
batch 1000: loss 0.012093
batch 1001: loss 0.007021
batch 1002: loss 0.067966
batch 1003: loss 0.015820
batch 1004: loss 0.012396
batch 1005: loss 0.016042
batch 1006: loss 0.116351
batch 1007: loss 0.000758
batch 1008: loss 0.027270
batch 1009: loss 0.006406
batch 1010: loss 0.002843
batch 1011: loss 0.044306
batch 1012: loss 0.007242
batch 1013: loss 0.002159
batch 1014: loss 0.102065
batch 1015: loss 0.111267
batch 1016: loss 0.082598
batch 1017: loss 0.021104
batch 1018: loss 0.032470
batch 1019: loss 0.043061
batch 1020: loss 0.014903
batch 1021: loss 0.002003
batch 1022: loss 0.054794
batch 1023: loss 0.032800
batch 1024: loss 0.008323
batch 1025: loss 0.016963
batch 1026: loss 0.008960
batch 1027: loss 0.094564
batch 1028: loss 0.131251
batch 1029: loss 0.006467
batch 1030: loss 0.0

batch 1312: loss 0.060244
batch 1313: loss 0.067346
batch 1314: loss 0.076009
batch 1315: loss 0.028006
batch 1316: loss 0.035503
batch 1317: loss 0.002072
batch 1318: loss 0.003509
batch 1319: loss 0.017509
batch 1320: loss 0.082751
batch 1321: loss 0.019234
batch 1322: loss 0.025234
batch 1323: loss 0.003753
batch 1324: loss 0.001105
batch 1325: loss 0.016919
batch 1326: loss 0.008187
batch 1327: loss 0.025794
batch 1328: loss 0.006450
batch 1329: loss 0.002986
batch 1330: loss 0.005158
batch 1331: loss 0.034232
batch 1332: loss 0.006687
batch 1333: loss 0.005691
batch 1334: loss 0.019134
batch 1335: loss 0.000686
batch 1336: loss 0.003653
batch 1337: loss 0.000441
batch 1338: loss 0.016664
batch 1339: loss 0.001323
batch 1340: loss 0.002739
batch 1341: loss 0.013946
batch 1342: loss 0.025621
batch 1343: loss 0.002394
batch 1344: loss 0.057863
batch 1345: loss 0.116565
batch 1346: loss 0.004630
batch 1347: loss 0.008359
batch 1348: loss 0.000463
batch 1349: loss 0.000545
batch 1350: 

...

batch 4493: loss 0.000196
batch 4494: loss 0.079844
batch 4495: loss 0.070919
batch 4496: loss 0.010822
batch 4497: loss 0.006619
batch 4498: loss 0.007078
batch 4499: loss 0.001293
batch 4500: loss 0.004607
batch 4501: loss 0.001588
batch 4502: loss 0.006253
batch 4503: loss 0.000709
batch 4504: loss 0.008511
batch 4505: loss 0.000394
batch 4506: loss 0.000609
batch 4507: loss 0.055946
batch 4508: loss 0.009441
batch 4509: loss 0.009951
batch 4510: loss 0.000121
batch 4511: loss 0.014644
batch 4512: loss 0.006197
batch 4513: loss 0.020604
batch 4514: loss 0.000461
batch 4515: loss 0.008086
batch 4516: loss 0.000609
batch 4517: loss 0.001118
batch 4518: loss 0.000189
batch 4519: loss 0.000519
batch 4520: loss 0.001057
batch 4521: loss 0.058177
batch 4522: loss 0.140568
batch 4523: loss 0.070378
batch 4524: loss 0.006865
batch 4525: loss 0.000621
batch 4526: loss 0.035141
batch 4527: loss 0.003145
batch 4528: loss 0.009030
batch 4529: loss 0.002681
batch 4530: loss 0.005788
batch 4531: 

batch 4812: loss 0.012432
batch 4813: loss 0.001063
batch 4814: loss 0.071332
batch 4815: loss 0.006801
batch 4816: loss 0.000792
batch 4817: loss 0.008125
batch 4818: loss 0.001438
batch 4819: loss 0.036207
batch 4820: loss 0.054517
batch 4821: loss 0.051542
batch 4822: loss 0.025905
batch 4823: loss 0.012820
batch 4824: loss 0.008942
batch 4825: loss 0.000934
batch 4826: loss 0.004191
batch 4827: loss 0.007620
batch 4828: loss 0.003041
batch 4829: loss 0.001078
batch 4830: loss 0.000363
batch 4831: loss 0.002787
batch 4832: loss 0.078908
batch 4833: loss 0.028587
batch 4834: loss 0.013354
batch 4835: loss 0.000077
batch 4836: loss 0.002755
batch 4837: loss 0.000497
batch 4838: loss 0.001689
batch 4839: loss 0.000188
batch 4840: loss 0.000481
batch 4841: loss 0.002158
batch 4842: loss 0.001183
batch 4843: loss 0.000373
batch 4844: loss 0.002683
batch 4845: loss 0.036049
batch 4846: loss 0.060304
batch 4847: loss 0.041398
batch 4848: loss 0.030132
batch 4849: loss 0.010149
batch 4850: 

batch 5131: loss 0.073572
batch 5132: loss 0.000068
batch 5133: loss 0.019977
batch 5134: loss 0.021680
batch 5135: loss 0.000235
batch 5136: loss 0.001211
batch 5137: loss 0.013856
batch 5138: loss 0.000032
batch 5139: loss 0.004481
batch 5140: loss 0.044881
batch 5141: loss 0.033330
batch 5142: loss 0.000543
batch 5143: loss 0.000331
batch 5144: loss 0.020129
batch 5145: loss 0.001665
batch 5146: loss 0.001027
batch 5147: loss 0.052587
batch 5148: loss 0.007371
batch 5149: loss 0.002619
batch 5150: loss 0.001896
batch 5151: loss 0.009849
batch 5152: loss 0.004663
batch 5153: loss 0.007575
batch 5154: loss 0.001468
batch 5155: loss 0.000565
batch 5156: loss 0.027774
batch 5157: loss 0.000142
batch 5158: loss 0.000883
batch 5159: loss 0.015005
batch 5160: loss 0.003987
batch 5161: loss 0.001162
batch 5162: loss 0.000071
batch 5163: loss 0.102582
batch 5164: loss 0.000052
batch 5165: loss 0.004703
batch 5166: loss 0.009810
batch 5167: loss 0.192318
batch 5168: loss 0.083870
batch 5169: 

batch 5448: loss 0.000326
batch 5449: loss 0.036363
batch 5450: loss 0.207414
batch 5451: loss 0.006892
batch 5452: loss 0.055324
batch 5453: loss 0.022946
batch 5454: loss 0.000192
batch 5455: loss 0.000090
batch 5456: loss 0.000226
batch 5457: loss 0.000152
batch 5458: loss 0.000093
batch 5459: loss 0.000173
batch 5460: loss 0.000290
batch 5461: loss 0.000031
batch 5462: loss 0.001021
batch 5463: loss 0.136333
batch 5464: loss 0.012553
batch 5465: loss 0.010739
batch 5466: loss 0.003542
batch 5467: loss 0.004889
batch 5468: loss 0.016698
batch 5469: loss 0.012770
batch 5470: loss 0.000153
batch 5471: loss 0.000100
batch 5472: loss 0.020742
batch 5473: loss 0.028244
batch 5474: loss 0.001180
batch 5475: loss 0.004726
batch 5476: loss 0.001039
batch 5477: loss 0.004917
batch 5478: loss 0.000564
batch 5479: loss 0.000814
batch 5480: loss 0.019318
batch 5481: loss 0.000697
batch 5482: loss 0.033357
batch 5483: loss 0.000710
batch 5484: loss 0.000937
batch 5485: loss 0.000164
batch 5486: 

batch 5764: loss 0.003094
batch 5765: loss 0.044686
batch 5766: loss 0.003017
batch 5767: loss 0.002516
batch 5768: loss 0.028398
batch 5769: loss 0.040970
batch 5770: loss 0.000166
batch 5771: loss 0.000724
batch 5772: loss 0.003442
batch 5773: loss 0.009829
batch 5774: loss 0.005004
batch 5775: loss 0.000130
batch 5776: loss 0.003282
batch 5777: loss 0.006599
batch 5778: loss 0.001411
batch 5779: loss 0.033949
batch 5780: loss 0.000614
batch 5781: loss 0.000524
batch 5782: loss 0.000837
batch 5783: loss 0.000408
batch 5784: loss 0.028224
batch 5785: loss 0.007485
batch 5786: loss 0.002352
batch 5787: loss 0.000613
batch 5788: loss 0.028041
batch 5789: loss 0.063975
batch 5790: loss 0.004688
batch 5791: loss 0.000653
batch 5792: loss 0.001420
batch 5793: loss 0.001650
batch 5794: loss 0.008174
batch 5795: loss 0.004946
batch 5796: loss 0.000467
batch 5797: loss 0.000665
batch 5798: loss 0.038920
batch 5799: loss 0.001884
batch 5800: loss 0.009601
batch 5801: loss 0.017684
batch 5802: 

In [4]:
"""模型的评估：tf.keras.metrics"""
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.989000
